In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_curve, auc

# new:
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch import optim

Take the data from Lino's preprocessing script

In [18]:
# # TAKEN DIRECTLY FROM COMBINED SCRIPT:

# easyshare = pd.read_stata('data/sharewX_rel8-0-0_easySHARE_stata/easySHARE_rel8-0-0.dta')

# illness_before = pd.read_stata("data/SHARE-ENV - Exposure to Environmental Hazards/illness_before_module_v01.dta")
# illness_during = pd.read_stata("data/SHARE-ENV - Exposure to Environmental Hazards/illness_during_module_v01.dta")
# job = pd.read_stata("data/SHARE-ENV - Exposure to Environmental Hazards/job_module_v01.dta")
# life = pd.read_stata("data/SHARE-ENV - Exposure to Environmental Hazards/life_module_v01.dta")
# young_age = pd.read_stata("data/SHARE-ENV - Exposure to Environmental Hazards/young_age_module_v01.dta")
# yearly = pd.read_stata("data/SHARE-ENV - Exposure to Environmental Hazards/yearly_module_v01.dta")

# df = pd.merge(easyshare, life, on=['mergeid', 'wave'], how='left')
# df = pd.merge(df, job, on=['mergeid'], how='left')

# #_____

# df_sorted = df.sort_values(by=['mergeid', 'wave'], ascending=[True, False])
# df_most_recent_wave_per_mergeid = df_sorted.drop_duplicates(subset='mergeid', keep='first') # this is the full dataset, we should be trying to run models on this default

# df_relevant = df_most_recent_wave_per_mergeid[df_most_recent_wave_per_mergeid.columns.drop(list(df.filter(regex='^euro')))]
# df_relevant = df_relevant[df_relevant.columns.drop(list(df_relevant.filter(regex='^dn')))]
# non_predictive_vars = [
#     'mergeid',    # Used for merging records, no predictive power
#     'hhid',       # Household identifier for tracking or grouping data
#     'coupleid',   # Links records of individuals within a household
#     'int_version',# Version of the questionnaire or interview format
#     'int_year',   # Year the interview was conducted, structural rather than predictive
#     'int_month',  # Month the interview was conducted, similar to int_year
#     'country',    # Country code, used for stratification or adjustments
#     'country_mod', # Modified country code, typically for data manipulation
#     'wavepart'   # Wave part, used for stratification or adjustments
# ]
# df_relevant = df_relevant[df_relevant.columns.drop(non_predictive_vars)]

# def replace_dash_with_na(df_relevant):
#     for column in df_relevant.columns:
#         if df_relevant[column].dtype == 'category':
#             # Replace entries containing '-' with NA
#             df_relevant[column] = df[column].apply(lambda x: pd.NA if '-' in str(x) else x)
#     return df_relevant

# df_relevant = replace_dash_with_na(df_relevant)

# #_____

# na_counts = df_relevant.groupby('wave').apply(lambda x: x.isnull().sum())
# # mean per number of abservation per wave
# na_counts['mean']= na_counts.mean(axis=1)
# na_counts['obs'] = df_relevant.groupby('wave').size()
# na_counts['avg_mean'] = na_counts['mean']/ na_counts['obs']
# na_counts['std'] = na_counts.std(axis=1)
# na_counts = na_counts.sort_values(by='avg_mean', ascending=False)
# df_sorted = df_relevant.sort_values(by=['wave'], ascending=[False])
# # drop all except wave 7 
# df_wave_7 = df_sorted[df_sorted['wave'] == 7]
# df_wave_7 = df_wave_7.drop(columns=['wave'])
# # df_most_recent_wave_per_mergeid = df_sorted.drop_duplicates(subset='mergeid', keep='first')

# na_counts = df_wave_7.isna().sum()

# na_counts_sorted = na_counts.sort_values(ascending=False)

# for column in df_wave_7.columns:
#     if df_wave_7[column].dtype == object:  # Check if the column data type is object
#         # Try converting the column to numeric
#         converted_column = pd.to_numeric(df_wave_7[column], errors='coerce')
#         # Check if the conversion did not introduce any new NaNs (i.e., all NaNs in the original are NaNs in the converted)
#         if converted_column.notna().equals(df_wave_7[column].notna()):
#             df_wave_7[column] = converted_column

# columns_to_drop = na_counts[na_counts > 20000].index

# df_dropped = df_wave_7.drop(columns=columns_to_drop)

# na_after_dr = df_dropped.isna().sum().sort_values(ascending=False)

# def process_categorical_columns(df):
#     for column in df.columns:
#         if df[column].dtype == 'category' or df[column].dtype == 'object':
#             # Convert category to string
#             df[column] = df[column].astype(str)

#             # Split the column on the first '.', and expand to new DataFrame
#             split_data = df[column].str.split('.', expand=True, n=1)

#             # If split_data has only one column, no '.' was found; skip processing
#             if split_data.shape[1] < 2:
#                 continue

#             # Clean up whitespace
#             split_data[1] = split_data[1].str.strip()

#             # Keep only rows where a split occurred (indicative of having a '.')
#             valid_splits = split_data[1].notna()

#             # Create dummy variables only for the valid name parts
#             if valid_splits.any():
#                 dummies = pd.get_dummies(split_data.loc[valid_splits, 1], prefix=column)
#                 # Merge these dummy variables back to the original DataFrame
#                 df = pd.concat([df, dummies], axis=1)

#             # Optionally, drop the original column
#             df.drop(column, axis=1, inplace=True)

#     return df

# df_final = process_categorical_columns(df_dropped)

# df_dropped.info()
# df_final.info()

In [19]:
# print("df_dropped dtypes:")
# print(df_dropped.info())
# #
# print("\ndf_final dtypes:")
# print(df_final.info())

In [20]:
X = pd.read_csv('data/X.csv')
y = pd.read_csv('data/y.csv')

In [29]:
print("X info:")
X.info()
print("\n y info:")
y.info()

X info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76822 entries, 0 to 76821
Columns: 495 entries, age to job_end_2019
dtypes: bool(259), float64(236)
memory usage: 157.3 MB

 y info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76822 entries, 0 to 76821
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   sphus   76822 non-null  object
dtypes: object(1)
memory usage: 600.3+ KB


In [36]:
y.head

<bound method NDFrame.head of               sphus
0           5. Poor
1           4. Fair
2           4. Fair
3           3. Good
4      2. Very good
...             ...
76817       4. Fair
76818       4. Fair
76819       3. Good
76820       4. Fair
76821       3. Good

[76822 rows x 1 columns]>

OK, need to convert this to numeric to make it into tensor (later will need to map it back to categorical to get meaningful answers)

In [41]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y['sphus'] = label_encoder.fit_transform(y['sphus'])
print(y)

       sphus
0          4
1          3
2          3
3          2
4          1
...      ...
76817      3
76818      3
76819      2
76820      3
76821      2

[76822 rows x 1 columns]


### From here is new

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [43]:
train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

# train is now 70% of the entire data set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_ratio, random_state = 10117)

# test is now 15% of the initial data set
# validation is now 15% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = test_ratio / (test_ratio + validation_ratio), random_state = 10117)

n_train = X_train.shape[0]
n_val = X_val.shape[0]
n_test = X_test.shape[0]
n = n_train + n_val + n_test

print((n_train / n), (n_val / n), (n_test / n)) 

0.6999947931582099 0.14999609486865742 0.15000911197313269


In [44]:
# training it: first need to remove all NaNs / impute

# Check for NaNs in the training data
if X_train.isna().any().any() or y_train.isna().any().any():
    print("NaNs in train data")

# Check for infinities in the training data
if (X_train == np.inf).any().any() or (y_train == np.inf).any().any():
    print("Infinities in train data")

# Impute & scale

In [45]:
# IMPUTING:

# Select non-numeric columns (booleans)
non_numeric_columns_train = X_train.select_dtypes(include=['object', 'bool']).columns
non_numeric_columns_test = X_test.select_dtypes(include=['object', 'bool']).columns
non_numeric_columns_val = X_val.select_dtypes(include=['object', 'bool']).columns

# Impute non-numeric columns with the string "missing"
X_train[non_numeric_columns_train] = X_train[non_numeric_columns_train].fillna("missing")
X_test[non_numeric_columns_test] = X_test[non_numeric_columns_test].fillna("missing")
X_val[non_numeric_columns_val] = X_val[non_numeric_columns_val].fillna("missing")

# Select numeric columns
numeric_columns_train = X_train.select_dtypes(include=[np.number]).columns
numeric_columns_test = X_test.select_dtypes(include=[np.number]).columns
numeric_columns_val = X_val.select_dtypes(include=[np.number]).columns

# Initialize the imputer for numeric columns
numeric_imputer = SimpleImputer(strategy='mean')

# Apply imputation to numeric columns in training data (fit & transform)
X_train[numeric_columns_train] = numeric_imputer.fit_transform(X_train[numeric_columns_train])

# Apply imputation to numeric columns in testing / val data using the *same* imputer (only transform)
X_test[numeric_columns_test] = numeric_imputer.transform(X_test[numeric_columns_test])
X_val[numeric_columns_test] = numeric_imputer.transform(X_val[numeric_columns_test])

In [46]:
# SCALING

# initialise
scaler = StandardScaler()

# Scale training data (fit and transform)
X_train_scaled = X_train.copy()
X_train_scaled[numeric_columns_train] = scaler.fit_transform(X_train[numeric_columns_train])

# Scale testing data (only transform)
X_test_scaled = X_test.copy()
X_test_scaled[numeric_columns_test] = scaler.transform(X_test[numeric_columns_test])

# Scale validation data (only transform)
X_val_scaled = X_val.copy()
X_val_scaled[numeric_columns_test] = scaler.transform(X_val[numeric_columns_test])

# Convert to Tensors

In [47]:
# Convert feature DataFrames to NumPy arrays and then to tensors
X_train_torch = torch.tensor(X_train_scaled.to_numpy().astype(np.float32))
X_val_torch = torch.tensor(X_val_scaled.to_numpy().astype(np.float32))
X_test_torch = torch.tensor(X_test_scaled.to_numpy().astype(np.float32))

# Convert label Series to tensors and unsqueeze to add an extra dimension
y_train_torch = torch.tensor(y_train.to_numpy().astype(np.float32)).unsqueeze(1)
y_val_torch = torch.tensor(y_val.to_numpy().astype(np.float32)).unsqueeze(1)
y_test_torch = torch.tensor(y_test.to_numpy().astype(np.float32)).unsqueeze(1)

In [ ]:


# DataLoader instances
train_dataset = TensorDataset(X_train_torch, y_train_torch) # makes them into joined Tensor
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True) # converts into a data loader, with 64 samples in a batch

val_dataset = TensorDataset(X_val_torch, y_val_torch)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [ ]:
# a training function:

def train_one_epoch(dataloader, model, loss_fn, optimizer):
    model.train()  
    total_loss = 0.0
    num_samples = 0

    for inputs, targets in dataloader:

        # forward pass: compute model output
        predictions = model(inputs)
        
        # compute loss
        loss = loss_fn(predictions, targets)

        # backward pass: compute  gradient of  loss wrt to model params
        optimizer.zero_grad()  # zero prev gradients
        loss.backward()  # backpropagation
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1) # for clipping gradients - needed later

        # update the model parameters - NB: calling the step function on an Optimizer makes an update to its params
        optimizer.step()

        # update total loss and the batch count
        total_loss += loss.item() * inputs.size(0) # this will depend on defined batchsize in dataloader?
        num_samples +=inputs.size(0) 

    average_loss = total_loss / num_samples
    print(f"Average loss: {average_loss:.4f}") # prints running avg
    return average_loss

In [ ]:
# a function to evaluate error on entire val / test set

def evaluate_model(dataloader, model, loss_fn):
    model.eval()  
    total_loss = 0.0
    num_samples = 0

    with torch.no_grad():  # disable gradient computation during evaluation
        for inputs, targets in dataloader:

            # forward pass: 
            predictions = model(inputs)
            
            # loss
            loss = loss_fn(predictions, targets)

            # aggregate the loss
            total_loss += loss.item() * inputs.size(0)  
            num_samples += inputs.size(0)
            
            # NB THE DIFFERENCE HERE IS NOT LEARNING NEW PARAMS IN EVAL MODE

    average_loss = total_loss / num_samples  
    print(f"Average loss over evaluation data: {average_loss:.4f}")

    return average_loss

In [ ]:
# defining basic NN model

class NeuralNetwork(nn.Module):
    def __init__(self, input_features, hidden_size):
        super(NeuralNetwork, self).__init__()
        self.hidden = nn.Linear(input_features, hidden_size)
        self.relu = nn.ReLU()
        self.output = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = self.hidden(x)
        x = self.relu(x)
        x = self.output(x)
        return x

In [ ]:

#  training model w/ diff sized hidden layers

def run_training2(train_loader, val_loader, epochs, hidden_size):
    
    model = NeuralNetwork(input_features=X_train_torch.shape[1], hidden_size=hidden_size) # added this inside of the function no we want to try diff values - need to pass hidden_size now it has multiple layers
    loss_fn = nn.MSELoss() # set these as fixed params
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # set these as fixed params
    
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        train_loss = train_one_epoch(train_loader, model, loss_fn, optimizer)
        val_loss = evaluate_model(val_loader, model, loss_fn)

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
    
    return train_losses, val_losses
        

# 10 hidden nodes
print("Training with 10 hidden nodes:")
train_losses_10, val_losses_10 = run_training2(train_loader=train_loader, 
                                             val_loader=val_loader,
                                             epochs = 20,
                                             hidden_size=10)

# 1000 hidden nodes
print("Training with 1000 hidden nodes:")
train_losses_1000, val_losses_1000 = run_training2(train_loader=train_loader, 
                                             val_loader=val_loader,
                                             epochs = 20,
                                             hidden_size=1000)

# plot
plt.figure(figsize=(12, 6))
plt.plot(train_losses_10, label='Training Loss (10 Nodes)')
plt.plot(val_losses_10, label='Validation Loss (10 Nodes)')
plt.plot(train_losses_1000, label='Training Loss (1000 Nodes)')
plt.plot(val_losses_1000, label='Validation Loss (1000 Nodes)')
plt.title('Training and Validation Losses')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# training IN THIS CASE A LINEAR MODEL using Adam


# step 1: get data ready SEE ABOVE


# Step 2: Initialize the Model, Loss Function, and Optimizer

model = SimpleLinearRegression(input_features=X_train_torch.shape[1])
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) 


# Step 3: Define Training Functions

def run_training(model, train_loader, val_loader, loss_fn, optimizer, epochs):
    train_losses = []
    val_losses = []
      
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1) # decreasing step sizes as had issue with NaNs
    
    for epoch in range(epochs):
        train_loss = train_one_epoch(train_loader, model, loss_fn, optimizer)
        val_loss = evaluate_model(val_loader, model, loss_fn)
        #scheduler.step()  # decreasing step sizes as had issue with NaNs
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
    
    return train_losses, val_losses


# Step 4: Training the Model and Collecting Errors

num_epochs = 500  # NB aritrary
train_losses, val_losses = run_training(model, train_loader, val_loader, loss_fn, optimizer, num_epochs)


# Step 5: Plotting  Learning Curves
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.axhline(y=mse_val, color='r', linestyle='--', label='Baseline Error (Validation MSE)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Learning Curves')
plt.legend()
plt.grid(True)
plt.show()
